<a href="https://colab.research.google.com/github/mucke2701/H-M-Recommendation/blob/main/HundMV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Open Drive-Connection to get all csv-files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip gdrive/My\ Drive/Kaggle/articles.csv.zip
!unzip gdrive/My\ Drive/Kaggle/customers.csv.zip
!unzip gdrive/My\ Drive/Kaggle/sample_submission.csv.zip
!unzip gdrive/My\ Drive/Kaggle/transactions_train.csv.zip

Archive:  gdrive/My Drive/Kaggle/articles.csv.zip
replace articles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  gdrive/My Drive/Kaggle/customers.csv.zip
replace customers.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  gdrive/My Drive/Kaggle/sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  gdrive/My Drive/Kaggle/transactions_train.csv.zip
replace transactions_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 198 kB 60.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ed72b8f2d824e627dda2426250a88480ac3125904e41a493f260db95034f5086
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
from pandas.io.json import json_normalize
from pprint import pprint
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
transactions = pd.read_csv('transactions_train.csv', dtype={'article_id': str})
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
#transactions.set_index('t_dat', inplace=True)


In [ ]:
customers = pd.read_csv('customers.csv')
articles = pd.read_csv('articles.csv', dtype={'article_id': str})

#Implicit

In [ ]:
transsept = transactions[transactions['t_dat']>='2020-09-01']

In [ ]:
ALL_USERS = customers['customer_id'].unique().tolist()
ALL_ITEMS = articles['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

transsept['user_id'] = transsept['customer_id'].map(user_map)
transsept['item_id'] = transsept['article_id'].map(item_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
!pip install implicit

     |████████████████████████████████| 18.5 MB 654 kB/s 


In [ ]:
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

row = transsept['user_id'].values
col = transsept['item_id'].values
data = np.ones(transsept.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


<1371980x105542 sparse matrix of type '<class 'numpy.float64'>'
	with 798269 stored elements in COOrdinate format>

In [ ]:
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(coo_train)

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:27: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 9.3 s, sys: 15.2 s, total: 24.5 s
Wall time: 20.8 s


In [ ]:
def to_user_item_coo(df):
    """ Turn a dataframe with transactions into a COO sparse items x users matrix"""
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo


def split_data(df, validation_days=7):
    """ Split a pandas dataframe into training and validation data, using <<validation_days>>
    """
    validation_cut = '2020-09-16'

    df_train = df[df['t_dat'] < validation_cut]
    df_val = df[df['t_dat'] >= validation_cut]
    return df_train, df_val

def get_val_matrices(df, validation_days=7):
    """ Split into training and validation and create various matrices
        
        Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
            csr_train: training data in CSR sparse format and as (users x items)
            csr_val:  validation data in CSR sparse format and as (users x items)
    
    """
    df_train, df_val = split_data(df, validation_days=validation_days)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension) 
    for <<iterations>> over matrices and validate with MAP@12
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [ ]:
matrices = get_val_matrices(transsept)

In [ ]:
%%time
best_map12 = 0
for factors in [100, 150 ,200]:
    for iterations in [3, 12, 14, 15, 20]:
        for regularization in [0.01]:
            map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"Best MAP@12 found. Updating: {best_params}")

Factors: 100 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00351
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 3, 'regularization': 0.01}
Factors: 100 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00409
Best MAP@12 found. Updating: {'factors': 100, 'iterations': 12, 'regularization': 0.01}
Factors: 100 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00407
Factors: 100 - Iterations: 15 - Regularization: 0.010 ==> MAP@12: 0.00407
Factors: 100 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.00406
Factors: 150 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00360
Factors: 150 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00392
Factors: 150 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00393
Factors: 150 - Iterations: 15 - Regularization: 0.010 ==> MAP@12: 0.00392
Factors: 150 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.00392
Factors: 200 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00382
Factors: 20

In [ ]:
coo_train = to_user_item_coo(transsept)
csr_train = coo_train.tocsr()

In [ ]:
def train(coo_train, factors=100, iterations=12, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

In [ ]:
best_params
model = train(coo_train, **best_params)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
def submit(model, csr_train, submission_name="submissions.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [ ]:
%%time
df_preds = submit(model, csr_train);

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0762846031 0568601006 0568597006 0568601044 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0112679048 0111609001 0111593001 0111586001 01...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0805000001 0804992014 0804992017 0912204001 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0112679048 0111609001 0111593001 0111586001 01...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0112679048 0111609001 0111593001 0111586001 01...


(1371980, 2)
CPU times: user 31min 4s, sys: 8min 1s, total: 39min 6s
Wall time: 20min 16s


#Weekly sales

In [ ]:
customers['gender']='d'

articlesindexandsection = pd.concat([articles['article_id'],pd.get_dummies(articles[['index_group_name','section_name']])], axis=1)
articlesindexandsection.head()

,article_id,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_Sport,section_name_Baby Boy,section_name_Baby Essentials & Complements,section_name_Baby Girl,section_name_Boys Underwear & Basics,...,section_name_Womens Lingerie,"section_name_Womens Nightwear, Socks & Tigh",section_name_Womens Premium,section_name_Womens Shoes,section_name_Womens Small accessories,"section_name_Womens Swimwear, beachwear",section_name_Womens Tailoring,section_name_Womens Trend,section_name_Young Boy,section_name_Young Girl
0,0108775015,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0108775044,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0108775051,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0110065001,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0110065002,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
transactions = transactions.merge(articlesindexandsection, how='inner')
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_Sport,...,section_name_Womens Lingerie,"section_name_Womens Nightwear, Socks & Tigh",section_name_Womens Premium,section_name_Womens Shoes,section_name_Womens Small accessories,"section_name_Womens Swimwear, beachwear",section_name_Womens Tailoring,section_name_Womens Trend,section_name_Young Boy,section_name_Young Girl
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2018-09-20,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,0663713001,0.049475,2,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2018-09-20,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,0663713001,0.050831,2,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,2018-09-20,6b7b10d2d47516c82a6f97332478dab748070f09693f09...,0663713001,0.050831,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2018-09-20,8ac137752bbe914aa4ae6ad007a9a0c5b67a1ab2b2d474...,0663713001,0.050831,2,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


#Creating Age Bins

In [ ]:
listBin = [-1, 19, 59, 119]
customers['age_bins'] = pd.cut(customers['age'], listBin)
customers['age_bins'].unique()

[(19.0, 59.0], NaN, (59.0, 119.0], (-1.0, 19.0]]
Categories (3, interval[int64, right]): [(-1, 19] < (19, 59] < (59, 119]]

In [ ]:
N = 3
listUniBins = customers['age_bins'].unique().tolist()
listUniBins

[Interval(19.0, 59.0, closed='right'),
 nan,
 Interval(59.0, 119.0, closed='right'),
 Interval(-1.0, 19.0, closed='right')]

#Implicit Gender, Sportiness and if customers has/or are kids

We loop through all the transactions check the articles index-name and section-name to assume the gender, sportiness and kids

In [ ]:
#Laufzeit 31 Min
implicitgender= {}
implicithaskids = {}
implicitsport = {}

for i, c in enumerate(transactions['customer_id']):
  if c not in implicitgender:
    implicitgender[c]={'m':0,'w':0,'p':0}
    implicithaskids[c]={'kidsitems':0, 'numberofitems':0}
    implicitsport[c]={'sportitems':0, 'numberofitems':0}
  if (transactions['index_group_name_Divided'][i]==1) or (transactions['index_group_name_Ladieswear'][i]==1) or (transactions['section_name_Ladies H&M Sport'][i]==1):
    implicitgender[c]['w']+=1
    implicitgender[c]['p']+=1
  elif (transactions['index_group_name_Menswear'][i]==1) or (transactions['section_name_Men H&M Sport'][i]==1):
    implicitgender[c]['m']+=1
    implicitgender[c]['p']+=1
  else:
    implicitgender[c]['p']+=1
  if (transactions['index_group_name_Baby/Children'][i]==1):
    implicithaskids[c]['kidsitems']+=1
    implicithaskids[c]['numberofitems']+=1
  else:
    implicithaskids[c]['numberofitems']+=1
  if (transactions['index_group_name_Sport'][i]==1):
    implicitsport[c]['sportitems']+=1
    implicitsport[c]['numberofitems']+=1
  else:
    implicitsport[c]['numberofitems']+=1


In [ ]:
# rows list initialization
rows = []
for cid, c_info in implicitgender.items():
  row = [cid, c_info['m'], c_info['p'], c_info['w']] 
  rows.append(row)

cgender = pd.DataFrame(data=rows,columns=['customer_id','m','p','w'])
cgender.head()

,customer_id,m,p,w
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,4,18,14
1,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,0,126,126
2,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,1,340,335
3,6b7b10d2d47516c82a6f97332478dab748070f09693f09...,10,109,99
4,8ac137752bbe914aa4ae6ad007a9a0c5b67a1ab2b2d474...,0,2,2


#Creating Ratios of implicit Attributes

In [ ]:
cgender['maleratio']=cgender['m']/cgender['p']
cgender[cgender['maleratio']>0.8]

cgender['femaleratio']=cgender['w']/cgender['p']
cgender[cgender['femaleratio']>0.8]

# rows list initialization
rows = []
for cid, c_info in implicitsport.items():
  row = [cid, c_info['sportitems'], c_info['numberofitems']] 
  rows.append(row)

csport = pd.DataFrame(data=rows,columns=['customer_id','sportitems','numberofitems'])
csport.head()

# rows list initialization
rows = []
for cid, c_info in implicithaskids.items():
  row = [cid, c_info['kidsitems'], c_info['numberofitems']] 
  rows.append(row)

ckids = pd.DataFrame(data=rows,columns=['customer_id','kidsitems','numberofitems'])
ckids.head()

csport['sportratio']=csport['sportitems']/csport['numberofitems']

ckids['kidsratio']=ckids['kidsitems']/ckids['numberofitems']

customers = customers.merge(cgender,on='customer_id' ,how='outer')
customers = customers.merge(csport,on='customer_id' ,how='outer')
customers = customers.merge(ckids,on='customer_id' ,how='outer')
customers

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,gender,age_bins,m,p,w,maleratio,femaleratio,sportitems,numberofitems_x,sportratio,kidsitems,numberofitems_y,kidsratio
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,d,"(19, 59]",1.0,21.0,15.0,0.047619,0.714286,0.0,21.0,0.000000,5.0,21.0,0.238095
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,d,"(19, 59]",1.0,86.0,81.0,0.011628,0.941860,1.0,86.0,0.011628,4.0,86.0,0.046512
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,d,"(19, 59]",4.0,18.0,14.0,0.222222,0.777778,1.0,18.0,0.055556,0.0,18.0,0.000000
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,d,"(19, 59]",0.0,2.0,2.0,0.000000,1.000000,2.0,2.0,1.000000,0.0,2.0,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,d,"(19, 59]",0.0,13.0,13.0,0.000000,1.000000,2.0,13.0,0.153846,0.0,13.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,NaN,NaN,ACTIVE,NONE,24.0,7aa399f7e669990daba2d92c577b52237380662f36480b...,d,"(19, 59]",1.0,51.0,50.0,0.019608,0.980392,4.0,51.0,0.078431,0.0,51.0,0.000000
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,NaN,NaN,ACTIVE,NONE,21.0,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...,d,"(19, 59]",2.0,84.0,75.0,0.023810,0.892857,0.0,84.0,0.000000,7.0,84.0,0.083333
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1.0,1.0,ACTIVE,Regularly,21.0,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...,d,"(19, 59]",5.0,45.0,40.0,0.111111,0.888889,3.0,45.0,0.066667,0.0,45.0,0.000000
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.0,1.0,ACTIVE,Regularly,18.0,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...,d,"(-1, 19]",0.0,7.0,7.0,0.000000,1.000000,0.0,7.0,0.000000,0.0,7.0,0.000000


#Conditions to make customer segmentation

In [ ]:
ctypes = [('male','sport'),('female','sport'),('mixed','kids'),('mixed','sport'),('male','mixed2'), ('female,mixed2'),('mixed','mixed2')]
conditions = [
    (customers['maleratio']>=0.9) & (customers['sportratio']>=0.5),
    (customers['femaleratio']>=0.9) & (customers['sportratio']>=0.5),
    (customers['kidsratio']>=0.5),
    (customers['sportratio']>=0.5),
    (customers['maleratio']>=0.9),
    (customers['femaleratio']>=0.9),
    (customers['maleratio']<0.9) & (customers['femaleratio']<0.9) & (customers['kidsratio']<0.5) & (customers['sportratio']<0.5)
    ]
####Values mr 0.75 fr 0.75 sr 0.3 kr 0.3 DELIVER 0.0223
####Values mr 0.90 fr 0.90 sr 0.5 kr 0.3 DELIVER 0.0226
N = 12

trans = pd.read_csv('transactions_train.csv', dtype={'article_id': str})

# Make Recommendation for each Customer Segmentation, with a linear function for weekly sales

In [ ]:
for i,ctype in enumerate(ctypes):
  for uniBin in listUniBins:
      df  = transsept.copy()
      if str(uniBin) == 'nan':
          dfCustomersTemp = customers[customers['age_bins'].isnull()]
      else:
          dfCustomersTemp = customers[(customers['age_bins'] == uniBin) & (conditions[i])]
      
      dfCustomersTemp = dfCustomersTemp.drop(['age_bins'], axis=1)
      
      df = df.merge(dfCustomersTemp[['customer_id', 'age']], on='customer_id', how='inner')
      print(f'The shape of scope transaction for {uniBin} is {df.shape}. \n')
            
      df['customer_id'] 
      df['t_dat'] = pd.to_datetime(df['t_dat'])
      last_ts = df['t_dat'].max()

      tmp = df[['t_dat']].copy()
      tmp['dow'] = tmp['t_dat'].dt.dayofweek
      tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
      tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

      df['ldbw'] = tmp['ldbw'].values
      
      weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
      weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
      
      df = df.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')
      
      weekly_sales = weekly_sales.reset_index().set_index('article_id')

      df = df.merge(
          weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
          on='article_id', suffixes=("", "_targ"))

      df['count_targ'].fillna(0, inplace=True)
      del weekly_sales
      
      df['quotient'] = df['count_targ'] / df['count']
      
      target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
      general_pred = target_sales.nlargest(N).index.tolist()
      general_pred = [ str(article_id) for article_id in general_pred]
      general_pred_str =  ' '.join(general_pred)
      del target_sales
      
      purchase_dict = {}

      tmp = df.copy()
      tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
      tmp['dummy_1'] = 1 
      tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1)

      a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
      tmp['y'] = a / np.sqrt(tmp['x']) + b * np.exp(-c*tmp['x']) - d

      tmp['dummy_0'] = 0 
      tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1)
      tmp['value'] = tmp['quotient'] * tmp['y'] 

      tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
      tmp = tmp.reset_index()

      tmp = tmp.loc[tmp['value'] > 0]
      tmp['rank'] = tmp.groupby("customer_id")["value"].rank("dense", ascending=False)
      tmp = tmp.loc[tmp['rank'] <= 12]

      purchase_df = tmp.sort_values(['customer_id', 'value'], ascending = False).reset_index(drop = True)
      purchase_df['prediction'] = purchase_df['article_id'].astype(str) + ' '
      purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
      purchase_df['prediction'] = purchase_df['prediction'].str.strip()
      purchase_df = pd.DataFrame(purchase_df)
      
      sub  = pd.read_csv('sample_submission.csv',
                              usecols= ['customer_id'], 
                              dtype={'customer_id': 'string'})
      
      numCustomers = sub.shape[0]
      
      sub = sub.merge(dfCustomersTemp[['customer_id', 'age']], on='customer_id', how='inner')

      sub['customer_id2'] = sub['customer_id']

      sub = sub.merge(purchase_df, left_on = 'customer_id2', right_on = 'customer_id', how = 'left',
                    suffixes = ('', '_ignored'))

      sub['prediction'] = sub['prediction'].fillna(general_pred_str)
      sub['prediction'] = sub['prediction'] + ' ' +  general_pred_str
      sub['prediction'] = sub['prediction'].str.strip()
      sub['prediction'] = sub['prediction'].str[:131]
      sub = sub[['customer_id', 'prediction']]
      sub.to_csv(f'submission_' + str(uniBin) + str(i)  + '.csv',index=False)
      print(f'Saved prediction for {uniBin} and {ctype}. The shape is {sub.shape}. \n')
      print('-'*50)
print('Finished.\n')
print('='*50)

The shape of scope transaction for (19.0, 59.0] is (40, 6). 

Saved prediction for (19.0, 59.0] and ('male', 'sport'). The shape is (485, 2). 

--------------------------------------------------
The shape of scope transaction for nan is (2931, 6). 

Saved prediction for nan and ('male', 'sport'). The shape is (15861, 2). 

--------------------------------------------------
The shape of scope transaction for (59.0, 119.0] is (11, 6). 

Saved prediction for (59.0, 119.0] and ('male', 'sport'). The shape is (67, 2). 

--------------------------------------------------
The shape of scope transaction for (-1.0, 19.0] is (1, 6). 

Saved prediction for (-1.0, 19.0] and ('male', 'sport'). The shape is (15, 2). 

--------------------------------------------------
The shape of scope transaction for (19.0, 59.0] is (6586, 6). 

Saved prediction for (19.0, 59.0] and ('female', 'sport'). The shape is (24741, 2). 

--------------------------------------------------
The shape of scope transaction for

In [ ]:
weeklytrend = pd.read_csv('submissionweeklytrend.csv')
for j, ctype in reversed(list(enumerate(ctypes))):
  for i, uniBin in enumerate(listUniBins):
      dfTemp  = pd.read_csv(f'submission_' + str(uniBin) + str(j) + '.csv')
      if i == 0 and j ==6:
          #weeklytrend = dfTemp
          weeklytrend = pd.merge(weeklytrend, dfTemp, on='customer_id', how='outer')
          weeklytrend = weeklytrend.fillna(-1)
          weeklytrend['prediction'] = weeklytrend.apply(
              lambda x: x['prediction_y'] if x['prediction_y'] != -1 else x['prediction_x'], axis=1)
          weeklytrend = weeklytrend.drop(columns=['prediction_y', 'prediction_x'])         
      else:
          weeklytrend = pd.merge(weeklytrend, dfTemp, on='customer_id', how='outer')
          weeklytrend = weeklytrend.fillna(-1)
          weeklytrend['prediction'] = weeklytrend.apply(
              lambda x: x['prediction_y'] if x['prediction_y'] != -1 else x['prediction_x'], axis=1)
          weeklytrend = weeklytrend.drop(columns=['prediction_y', 'prediction_x'])

assert weeklytrend.shape[0] == numCustomers, f'The number of dfSub rows is not correct. {weeklytrend.shape[0]} vs {numCustomers}.'

weeklytrend.to_csv(f'submissionultibins.csv', index=False)
print(f'Saved submission.csv.')

Saved submission.csv.


#LGBMRanker in Work

In [ ]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)